In [16]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA3_256
from Crypto.PublicKey import RSA
import yaml
import base64

In [17]:
# Some helpful functions

def sign_with_rsa_pkcs1v15_and_sha3_256(private_key_pem, data_bytes):
    # Import the private key from pem format
    rsa_private_key = RSA.import_key(private_key_pem)
    # Sign the data
    signature = pkcs1_15.new(rsa_private_key).sign(SHA3_256.new(data_bytes))
    # Return the signature
    return signature


def verify_signature_with_rsa_pkcs1v15_and_sha3_256(public_key_pem, data_bytes, signature):
    # Import the public key from pem format
    rsa_public_key = RSA.import_key(public_key_pem)
    # Verify the signature
    try:
        pkcs1_15.new(rsa_public_key).verify(SHA3_256.new(data_bytes), signature)
        return True
    except (ValueError, TypeError):
        return False


def encode_bytes_to_base64_string(bytes):
    return base64.b64encode(bytes).decode('utf-8')


def decode_base64_string_to_bytes(base64_string):
    return base64.b64decode(base64_string)


In [23]:
# Generate RSA key pair for entities
def generate_public_private_key_pair(filename):
    # Generate a new RSA key pair with a key length of 2048 bits
    key = RSA.generate(2048)

    # Extract the public and private keys as byte strings
    private_key = key.export_key()
    public_key = key.publickey().export_key()

    with open(f'{filename}.pri.pem', 'w') as f:
        f.write(private_key.decode('utf-8'))
    
    with open(f'{filename}.pub.pem', 'w') as f:
        f.write(public_key.decode('utf-8'))

# Generate a new RSA key pair with a key length of 2048 bits for the CA
generate_public_private_key_pair('temp-cert/client-c')

In [26]:
# Generate the certificates using entities
def generate_certificate(ID, CA_ID):
    # Load the private key of the CA
    private_key_file = f'temp-cert/{CA_ID}.pri.pem'
    with open(private_key_file, 'r') as f:
        private_key = f.read()

    # read the public key of the ID
    with open(f'temp-cert/{ID}.pub.pem', 'rb') as f:
        public_key_bytes = f.read()

    # Encode the public key to base64
    public_key_base64 = encode_bytes_to_base64_string(public_key_bytes)

    # Total string
    total_string = ID + public_key_base64
    total_bytes = total_string.encode('utf-8')

    # Create a signature
    signature = sign_with_rsa_pkcs1v15_and_sha3_256(private_key, total_bytes)
    signature_base64 = encode_bytes_to_base64_string(signature)
    
    # Make the dictionary
    certificate = {
        'name': ID,
        'public-key-base64': public_key_base64,
        'signature-base64': signature_base64
    }

    # Save the certificate to a file in json format
    with open(f'temp-cert/{ID}.crt', 'w') as f:
        yaml.dump(certificate, f, indent=4)

generate_certificate('client-c', 'ca')


In [27]:
# Useful for validating certificates

def validate_certificate(cert_string):
    # Get the public key of the CA
    with open('temp-cert/ca.pub.pem', 'r') as f:
        public_key_ca = f.read()
    # Validate the certificate
    certificate = yaml.load(cert_string, Loader=yaml.SafeLoader)
    # Extract the public key and signature from the certificate
    name = certificate['name']
    public_key_base64 = certificate['public-key-base64']
    signature_base64 = certificate['signature-base64']
    # Decode the public key and signature from base64
    signature = decode_base64_string_to_bytes(signature_base64)
    # generate total string
    total_string = name + public_key_base64
    total_bytes = total_string.encode('utf-8')
    # Verify the signature
    return verify_signature_with_rsa_pkcs1v15_and_sha3_256(public_key_ca, total_bytes, signature)


def validate_certificate_file(filename):
    with open(filename, 'r') as f:
        cert_string = f.read()
    return validate_certificate(cert_string)


print(validate_certificate_file('temp-cert/server-s.crt'))
print(validate_certificate_file('temp-cert/client-a.crt'))
print(validate_certificate_file('temp-cert/client-b.crt'))
print(validate_certificate_file('temp-cert/client-c.crt'))

True
True
True
True
